In [1]:
import sys, os
import serial
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import win32com.client
from astropy.io import fits
import threading
import nbimporter
from picoammeter_control import (
    send_cmd,
    query,
    init_cmds
)
from camera_control_ascom import (
    take_ramp,
    camera_init,
    populate_header
)
sys.path.insert(0, os.path.abspath('.'))
sys.path.insert(0, os.path.abspath('filter_wheel_control'))
from filter_wheel_control.wheel_controller import (
    list_devices,
    open_device,
    get_position,
    get_position_count,
    set_position,
    close_device
)
from data_helper_functions import (
    init_wheels,
    close_wheels,
    camera_init,
    picoam_stability,
    picoam_worker_exp,
    camera_with_picoam,
    collect_var_exp,
    var_exp_graphs,
    capture_frame,
    measure_ptc_point,
    get_gain,
    picoam_worker_filters,
    camera_with_filters,
    collect_var_filters,
    compare_exp_and_filter,
    collect_bias_frames
)

In [2]:
camera_init()
cam_name = "ASCOM.ASICamera2_2.Camera"
save_path= "C:/Users/Jonah/fits_imgs"

You selected: 


In [ ]:
min_exp = 0.0001
max_exp = 2.5

In [ ]:
# picoam_stability(read_time = .5, save = save_path, port='COM5')

In [ ]:
init_wheels(print_stuff=False)
close_wheels()

In [ ]:
e_per_adu = get_gain(cam_name="ASCOM.ASICamera2_2.Camera", n_frames=3, print_stuff=True, exp_list=[min_exp, min_exp*10, min_exp*100, min_exp*200, min_exp*500, min_exp*1000])

In [ ]:
I_flat, Q_ramp, mean_signal, std_signal, bit_depth, exp_times, exp_low, exp_high, exp_50 = (collect_var_exp(cam_name=cam_name, num_exp=50, num_frames=10, exp_low = min_exp, exp_high = max_exp, save_path=save_path))
var_exp_graphs(mean_signal, std_signal, bit_depth, exp_times, exp_low, exp_high, save=True, e_per_adu=e_per_adu, min_percent=0.01, max_percent= 0.95, percent_change=False)

In [ ]:
signals_f, Q_filt = collect_var_filters(save_path=save_path, cam_name=cam_name, exp_50=exp_50, num_frames=10)

In [ ]:
bias_level = collect_bias_frames(cam_name = cam_name,num_frames=500, exp_low=min_exp, save_path=save_path)
print("bias level: ", bias_level)

In [ ]:
compare_exp_and_filter(e_per_adu = e_per_adu, Q_ramp=Q_ramp, Q_filt=Q_filt, signals_f=signals_f, mean_signal=mean_signal, bias_level=(bias_level-100), save=True)
